In [133]:
def read_initial_node():
    global initial_node 
    initial_node = int(input('Enter Start node : '))

def read_goal_node():
    global goal_node
    goal_node = int(input('Enter goal node : '))
    
def read_number_of_nodes():
    global nodes_count
    nodes_count = int(input('Enter number of nodes : '))

    

In [168]:
def get_adjacent_nodes(node,visited_array):
    adj_nodes = []
    for adj_node in range(nodes_count):
        if transition_matrix[node,adj_node] not in (0,float('inf')):
            if visited_array[adj_node]==0:
                adj_nodes.append(adj_node)
    return adj_nodes

def is_goal_node(node):
    if node==goal_node:
        return True
    return False

In [154]:
def cost(path):
    cost=0
    for index in range(len(path)-1):
        cost+=transition_matrix[path[index],path[index+1]]
    return cost
def display_path(path):
    delim = "->"
    print('path :',str(delim.join(map(str, path))))

In [172]:
def dfs_algorithm(visited_array,dfs_stack,path):
    
    node = dfs_stack[-1]
    visited_array[node]=1
    path.append(node)
    
    if is_goal_node(node):
        return "Node found"
    
    else:
        adj_nodes = []
        adj_nodes = get_adjacent_nodes(node,visited_array)
        dfs_stack.pop()
        if len(adj_nodes) > 0:
            dfs_stack+=adj_nodes
            dfs_algorithm(visited_array,dfs_stack,path)
        else:
            if len(dfs_stack)>0:
                dfs_algorithm(visited_array,dfs_stack,path)
            else:
                return "Node not found"

def dfs():
    visited_array = np.array(np.zeros(nodes_count))
    dfs_stack = [initial_node]
    path=[]
    dfs_algorithm(visited_array,dfs_stack,path)
    display_path(path)
    print('cost : ',cost(path))
    print(visited_array)
    print(dfs_stack)
    return cost(path)

In [9]:
import numpy as np

def read_data():
    read_number_of_nodes()
    read_graph()

read_data()

Enter number of nodes : 8
Enter edges from node 0 : 1:15,2:10,3:17,6:5
Enter edges from node 1 : 3:12
Enter edges from node 2 : 6:7
Enter edges from node 3 : 4:2,5:10,7:4 
Enter edges from node 4 : 
Enter edges from node 5 : 7:11
Enter edges from node 6 : 7:25
Enter edges from node 7 : 


A=0,B=1,C=2,D=3,E=4,F=5,G=6,H=7
A->1:15,2:10,3:17,6:5
B->3:12
C->6:7
D->4:2,5:10,7:4
E->
F->7:11
G->7:25
H->

In [173]:
def dfs():
    visited_array = np.array(np.zeros(nodes_count))
    dfs_stack = [initial_node]
    path=[]
    dfs_algorithm(visited_array,dfs_stack,path)
    display_path(path)
    print(visited_array)
    print(dfs_stack)
    print('cost : ',cost(path))
    return cost(path)

In [174]:
read_initial_node()
read_goal_node()
dfs()

Enter Start node : 0
Enter goal node : 6
path : 0->6
[1. 0. 0. 0. 0. 0. 1. 0.]
[1, 2, 3, 6]
cost :  5.0


5.0

In [122]:
def get_hueristic_values(g_node):
    hue_values = []
    goal_node = g_node
    for node in range(nodes_count):
        print(goal_node)
        initial_node = node
        hue_values.append(dfs())
    return hue_values
    

In [123]:
print(get_hueristic_values(4))

4
path : 0->1->3->4->3->5
[1. 1. 0. 1. 1. 1. 0. 0.]
[0, 1, 3, 5]
cost :  41.0
4
path : 0->1->3->4->3->5
[1. 1. 0. 1. 1. 1. 0. 0.]
[0, 1, 3, 5]
cost :  41.0
4
path : 0->1->3->4->3->5
[1. 1. 0. 1. 1. 1. 0. 0.]
[0, 1, 3, 5]
cost :  41.0
4
path : 0->1->3->4->3->5
[1. 1. 0. 1. 1. 1. 0. 0.]
[0, 1, 3, 5]
cost :  41.0
4
path : 0->1->3->4->3->5
[1. 1. 0. 1. 1. 1. 0. 0.]
[0, 1, 3, 5]
cost :  41.0
4
path : 0->1->3->4->3->5
[1. 1. 0. 1. 1. 1. 0. 0.]
[0, 1, 3, 5]
cost :  41.0
4
path : 0->1->3->4->3->5
[1. 1. 0. 1. 1. 1. 0. 0.]
[0, 1, 3, 5]
cost :  41.0
4
path : 0->1->3->4->3->5
[1. 1. 0. 1. 1. 1. 0. 0.]
[0, 1, 3, 5]
cost :  41.0
[41.0, 41.0, 41.0, 41.0, 41.0, 41.0, 41.0, 41.0]


In [85]:
def dfs_algorithm1(visited_array,dfs_stack,path):
    if is_goal_node(dfs_stack[-1],visited_array,path):
        dfs_stack.pop()
        adj_node = get_adjacent_node_dfs(dfs_stack[-1],visited_array)
        print(adj_node)
        if adj_node != -1:
            dfs_stack.append(adj_node)
            visited_array = np.array(np.zeros(nodes_count))
            for v in dfs_stack:
                visited_array[v]=1
            print('x :',visited_array)
            print('x :',dfs_stack)
            print('y :',path)
            dfs_algorithm1(visited_array,dfs_stack,path)
            print("top of the stack :",dfs_stack[-1])
        return "node found"
    else:
        adj_node = get_adjacent_node_dfs(dfs_stack[-1],visited_array)
        if adj_node != -1:
            dfs_stack.append(adj_node)
            dfs_algorithm1(visited_array,dfs_stack,path)
        else:
            print('backtrack : ',dfs_stack)
            dfs_stack.pop()
            if len(dfs_stack)>0:
                dfs_algorithm1(visited_array,dfs_stack,path)
            else:
                return "Node not found"

In [148]:
visited_array = np.array(np.zeros(nodes_count))
dfs_stack = [0]

In [151]:
get_adjacent_nodes(0,visited_array)

[1, 2, 3, 6]

In [101]:
read_goal_node()
goal_node = 5

Enter goal node : 3


In [129]:
x=[1,2,3]
y=[4,5,6]
x+=y
x
z=[]


In [132]:
len(z)

0